learn from [here](https://huggingface.co/blog/ngxson/make-your-own-rag)

Here is a simple RAG architecture 
![](../images/simple-RAG-architecture.svg)

**Loading the datasets**

In [1]:
dataset = []
with open('../data/cat-facts.txt', 'r') as file: 
    dataset = file.readlines()
    print(f"Loaded {len(dataset)} entries")

Loaded 150 entries


**Implement the vector database**

In [2]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

Now, let's implement the vector database.

We will use the embedding model from ollama to convert each chunk into an embedding vector, then store the chunk and its corresponding vector in a list.

Here is an example function to calculate the embedding vector for a given text:

In [3]:
# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
    embedding = ollama.embed(
        model = EMBEDDING_MODEL, 
        input = chunk
    )['embeddings'][0]
    VECTOR_DB.append((chunk, embedding))

Lets consider each line in the dataset as a chunk for simplicity.

In [4]:
for i, chunk in enumerate(dataset): 
    add_chunk_to_database(chunk)
    print(f"Added chunk {i + 1}/{len(dataset)} to the database")

Added chunk 1/150 to the database
Added chunk 2/150 to the database
Added chunk 3/150 to the database
Added chunk 4/150 to the database
Added chunk 5/150 to the database
Added chunk 6/150 to the database
Added chunk 7/150 to the database
Added chunk 8/150 to the database
Added chunk 9/150 to the database
Added chunk 10/150 to the database
Added chunk 11/150 to the database
Added chunk 12/150 to the database
Added chunk 13/150 to the database
Added chunk 14/150 to the database
Added chunk 15/150 to the database
Added chunk 16/150 to the database
Added chunk 17/150 to the database
Added chunk 18/150 to the database
Added chunk 19/150 to the database
Added chunk 20/150 to the database
Added chunk 21/150 to the database
Added chunk 22/150 to the database
Added chunk 23/150 to the database
Added chunk 24/150 to the database
Added chunk 25/150 to the database
Added chunk 26/150 to the database
Added chunk 27/150 to the database
Added chunk 28/150 to the database
Added chunk 29/150 to the dat

**Implement the retrieval function**

let's implement the retrieval function that takes a query and returns the top N most relevant chunks based on cosine similarity. We can imagine that the higher the cosine similarity between the two vectors, the "closer" they are in the vector space. This means they are more similar in terms of meaning.

Here is an example function to calculate the cosine similarity between two vectors:

In [5]:
def cosine_similarity(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

Now lets implement the retrieval function

In [6]:
def retrieve(query, top_n=3):
    query_embedding = ollama.embed(
        model = EMBEDDING_MODEL,
        input = query
    )["embeddings"][0]

    # temporary list to store (chunk, similarity) pairs
    similarities = []
    
    for chunk, embedding in VECTOR_DB: 
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))

    # sort by similarity in descending order, becaase higher similarity means more relevant chunks 
    similarities.sort(key=lambda x: x[1], reverse=True)

    # finally, return the top N most relevant chunks 
    return similarities[:top_n]

**Generation Phase** 

Here, he chatbot will generate a response based on the retrieved knowledge from the step above. This is done by simply add the chunks into the prompt that will be taken as input for the chatbot.

For instance, a prompt can be constructed as follows:

In [7]:
# Let just ask a question 
input_query = input("Ask me a question: ")
retrieved_knowledge = retrieve(input_query)

# Let the LLM Answer it 
print("Retrieved knowledge:")
for chunk, similarity in retrieved_knowledge: 
    print(f" - (similarity: {similarity:.2f}) {chunk}")

Ask me a question:  tell me about cat speed


Retrieved knowledge:
 - (similarity: 0.78) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (similarity: 0.66) Cats are extremely sensitive to vibrations. Cats are said to detect earthquake tremors 10 or 15 minutes before humans can.

 - (similarity: 0.66) Researchers are unsure exactly how a cat purrs. Most veterinarians believe that a cat purrs by vibrating vocal folds deep in the throat. To do this, a muscle in the larynx opens and closes the air passage about 25 times per second.



Lets now use  the `ollama` to generate the response. In this example, we will use `instruction_prompt` as system message:

In [10]:
# Lets instruct the bot  
instruction_prompt = f""" You are a helpful chatbot. 
Use only the following pieces of context to answer the question. Don't make any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
"""

In [11]:
stream = ollama.chat(
    model = LANGUAGE_MODEL, 
    messages = [
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query}
    ], 
    stream = True
)

# print the response from the chatbot in real-time 
print('Chatbot response:')
for chunk in stream: 
    print(chunk['message']['content'], end='', flush=True)

Chatbot response:
According to the context, a cat can travel at approximately 31 mph (49 km) over a short distance.